In [2]:
from pathlib import Path
import sqlite3
import pandas as pd

ROOT = Path.cwd().parents[0]  # notebooks/ の1つ上 = keiba-model/
db_path = ROOT / "data" / "keiba.db"
print("DB_PATH:", db_path)

conn = sqlite3.connect(db_path)

df_feat = pd.read_sql_query(
    "SELECT * FROM v_race_features WHERE race_id = 'R2025KYT11' ORDER BY finish_rank;",
    conn,
)

conn.close()

df_feat.head(20)


DB_PATH: c:\Users\r_bas\keiba-model\data\keiba.db


,race_id,date,course_id,venue_id,course_name,course_surface,course_track_type,straight_len,slope_max,race_no,...,finish_rank,finish_status,finish_time_sec,odds,popularity,weight,weight_diff,last_3f,margin_sec,prize
0,R2025KYT11,2025-10-26,KYT_T_OUT,KYT,京都 芝 外回り,turf,outer,None,None,11,...,1,OK,184.0,None,1,456,-12,35.0,None,None
1,R2025KYT11,2025-10-26,KYT_T_OUT,KYT,京都 芝 外回り,turf,outer,None,None,11,...,2,OK,184.3,None,2,508,-2,35.2,None,None
2,R2025KYT11,2025-10-26,KYT_T_OUT,KYT,京都 芝 外回り,turf,outer,None,None,11,...,3,OK,184.4,None,13,480,8,35.7,None,None
3,R2025KYT11,2025-10-26,KYT_T_OUT,KYT,京都 芝 外回り,turf,outer,None,None,11,...,4,OK,184.4,None,5,524,8,35.2,None,None
4,R2025KYT11,2025-10-26,KYT_T_OUT,KYT,京都 芝 外回り,turf,outer,None,None,11,...,5,OK,184.5,None,9,482,-4,35.5,None,None
5,R2025KYT11,2025-10-26,KYT_T_OUT,KYT,京都 芝 外回り,turf,outer,None,None,11,...,6,OK,184.5,None,8,492,6,35.1,None,None
6,R2025KYT11,2025-10-26,KYT_T_OUT,KYT,京都 芝 外回り,turf,outer,None,None,11,...,7,OK,184.6,None,15,438,-4,35.3,None,None
7,R2025KYT11,2025-10-26,KYT_T_OUT,KYT,京都 芝 外回り,turf,outer,None,None,11,...,8,OK,184.7,None,6,478,2,35.2,None,None
8,R2025KYT11,2025-10-26,KYT_T_OUT,KYT,京都 芝 外回り,turf,outer,None,None,11,...,9,OK,184.8,None,10,484,-6,36.0,None,None
9,R2025KYT11,2025-10-26,KYT_T_OUT,KYT,京都 芝 外回り,turf,outer,None,None,11,...,10,OK,185.1,None,17,452,0,35.9,None,None


In [ ]:
df = df_feat.copy()

# 目的変数
df["in_top3"] = (df["finish_rank"] <= 3).astype(int)

df[["horse_name", "finish_rank", "popularity", "in_top3"]].head(10)


# 目的変数

df["in_top3"]

SyntaxError: invalid syntax (1617095869.py, line 1)

In [ ]:
features = [
    "popularity",
    "weight", "weight_diff",
    "last_3f",
    "finish_time_sec",
]

df_ml = df[features + ["in_top3"]].dropna()

X = df_ml[features]
y = df_ml["in_top3"]

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = RandomForestClassifier()
model.fit(X_train, y_train)

proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, proba)

auc
